# Lab3b Magnetometer Data Analysis: Part 2
Contents of this notebook are developed by Dogacan Su Ozturk for the GEOS 419 Lecture. 

# This notebook

1. Loading EZIE Magnetometer Data from the Class Deployment
   - Reading the data
   - Removing erroneous data
   - Removing outliers
   - Searching for other artificial signatures
2. Downloading GIMA and USGS Magnetometer Data
   - Downloading data for Poker Flat (GIMA) and College (USGS) sites
3. Comparing EZIE measurements with the science grade magnetometers
   - Plotting data with Matplotlib
   - Analyzing the warm-up period
   - Removing baseline
   - Labelling and saving figures
4. Investigating the spatial extent of the geomagnetic perturbations
   - Comparison along a similar longitude
   - Comparison along similar latitudes


### 1. Loading EZIE Magnetometer Data from the Class Deployment
We have completed the first part of this laboratory class and we are now able to read and clean EZIE magnetometer data. Let's import the libraries we have used before and analyze the data from the deployment.

In [ ]:
# Essential Python libraries for this exercise
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
myFmt = mdates.DateFormatter('%D/%m %H:%M')
%matplotlib widget 

# Libraries for unzipping eziemag data
from ezie_mag_helper_functions import unzip_ezie_files

# Libraries for merging eziemag data
from ezie_mag_helper_functions import merge_eziemag_data

We can now load the data from EZIE magnetometer kits. Remember these files are now in a different path. Similar to the first part of this laboratory, we will start with the unzip_ezie_files function. Once you download the data using the eziemag.local interface, you will notice the files are zipped in a specific format. The functions you will use are geared towards 60 second averaged files. You will need to develop your own helper functions or modify the existing functionality by changing the folderpath if you wish to use 1 second data. In the examples below, we are using the unzip_ezie_files function. This function takes three parameters, the first one being the keyword string assigned to the magnetometer, the second is a string of folderpath, and the third is a logical flag if you want to copy all individual daily measurements unzipped into one big folder. By design, this folder will be placed in the folderpath, with a name appending the keyword with '_merged'.

In [ ]:
geos1_filepath = unzip_ezie_files('geos1','./GEOS_419_1/Deployment_01_20250205_17/', merge=True)
geos2_filepath = unzip_ezie_files('geos2','./GEOS_419_2/Deployment_01_20250205_17/', merge=True)
geos3_filepath = unzip_ezie_files('geos3','./GEOS_419_3/Deployment_01_20250205_17/', merge=True)

The code cell when executed will return print statements to help users locate filepath errors. Now that a new folder for unzipped files are generated, we can start reading the data. We will use the merge_eziemag_data function to read all the measurement data obtained. This function only needs the folderpath, which the unzip_eziemag_data provides. This function, calls another function named read_and_parse_ezie_mag_data. If user wants to read and parse single day data, they can pass the folderpath for this folder instead. 

We can now pass the path to function to generate a Pandas Dataframe for measurements from each different EZIE MAG kit. 

In [ ]:
geos1_filepath

In [ ]:
geos1_data = merge_eziemag_data(geos1_filepath)
geos2_data = merge_eziemag_data(geos2_filepath)
geos3_data = merge_eziemag_data(geos3_filepath)

You probably noticed that the number of files in folders are uneven. What does that mean? Our data might be corrupted or simply the few minute difference we plugged the kits caused different amount of files. Let's use some of the pandas techniques to find out.

First, let's look at the few first rows of each DataFrame to see if the data is read correctly.

In [ ]:
geos1_data.head()

In [ ]:
geos2_data.head()

In [ ]:
geos3_data.head()

Now, let's analyze each DataFrame to see if there is even numbers of data in all columns using .describe() functionality. 

In [ ]:
geos1_data.describe()

In [ ]:
geos2_data.describe()

In [ ]:
geos3_data.describe()

Is there anything going on? Now let's check if there is any corrupted data by looking for NaNs or NaTs in the DataFrame.

In [ ]:
for key in geos1_data.keys():
    print('Key: ', key, 'Amount of NaNs: ', geos1_data[key].isnull().sum())

In [ ]:
for key in geos2_data.keys():
    print('Key: ', key, 'Amount of NaNs: ', geos2_data[key].isnull().sum())

In [ ]:
for key in geos3_data.keys():
    print('Key: ', key, 'Amount of NaNs: ', geos3_data[key].isnull().sum())

We have some corrupted data in geos2 and geos3. Let's remove these data points. 

In [ ]:
geos2_data.dropna(inplace=True)
geos3_data.dropna(inplace=True)

In [ ]:
for key in geos2_data.keys():
    print('Key: ', key, 'Amount of NaNs: ', geos2_data[key].isnull().sum())

Now let's see if we have any outliers. How can we do that?

In [ ]:
for key in geos3_data.keys():
    print('Key: ', key, 'Min: ', geos3_data[key].min(), 'Max: ', geos3_data[key].max())

We got rid of the corrupted data points and didn't spot any outliers. Now let's go ahead and see if we can spot any further information.

In [ ]:
geos1_data['Bx'].plot()

What do you think the duration of the warm-up period? Is it the same for all three magnetometers?

### 2. Downloading GIMA and USGS Magnetometer Data
Next step to investigate our EZIE magnetometer measurements is to understand their accuracy. For this, we need to download magnetometer measurements for scientific grade instruments. There are two magnetometer stations one from Geophysical Institute Magnetometer Array (GIMA) at Poker Flat, AK and another from U.S. Geological Survey at College, AK. The widely used code names for these magnetometers are 'PKR'/'POKR' and 'CMO' respectively. We will use the SPEDAS package to download the data for these two sites.

In [ ]:
# Libraries for downloading professional grade magnetometer measurements
import pyspedas 

Then we will need to set a time range, using start and stop times. These times are passed as strings to the pyspedas.themis.gmag library. You can instead use any start_time and end_time in datetime.datetime format. Here, I am using the first and last time entries in the geos1_data to match the data with our measurements.

In [ ]:
#set start and end time for data analysis
start_time = pd.to_datetime(geos1_data.timeString.iloc[0])
end_time = pd.to_datetime(geos1_data.timeString.iloc[-1])
trange = [start_time.strftime('%Y-%m-%d/%H:%M:%S'), end_time.strftime('%Y-%m-%d/%H:%M:%S')]

Let's first download the data from Poker Flat station. We can pass sites=['pokr', 'cmo'] and download data simultaneously but for demonstraton purposes, I will download and assign data separately.

In [ ]:
poker_data = pyspedas.themis.gmag(trange=trange, sites=['pokr'], 
                                level='l2', time_clip=True, notplot=True)

Let's check what the data looks like.

In [ ]:
poker_data

The data is read into a dictionary. This is different than the Pandas Dataframe. Here the 'x' is unixtime, 'y' is the magnetometer measurements, with each column representing Bx, By, Bz. Now, we will do the same for College site.

In [ ]:
college_data = pyspedas.themis.gmag(trange=trange, sites=['cmo'], 
                                level='l2', time_clip=True,notplot=True)

Finally, we need to convert the UnixTime to UT time.

In [ ]:
poker_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in poker_data['thg_mag_pokr']['x']]
college_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in college_data['thg_mag_cmo']['x']]

### 3. Comparing EZIE measurements with the science grade magnetometers
Now that we have our own EZIE magnetometer measurements and science grade measurements, let's plot and compare the results. Below I will provide lines of code required to plot different components of the data set. Feel free to modify them as you see fit for your purposes. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx'], color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx'], color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx'], color='#FF858D', label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Magnetic Field Measurements - X Component', fontsize=16)

# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()


What are some of the things we can notice right away? Do all three magnetometers have the same warm up period? Do they all have the same values?

In [ ]:
geos1_data.Bx.head(10), geos2_data.Bx.head(10), geos3_data.Bx.head(10)

We can see what we regard as "contaminated period" has different durations for each magnetometer kit. We can either remove the first few minutes from all measurements or remove values based on variability of this duration.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
ax.plot(geos1_data['Bx'], color='#FF9FE5', marker='o', label='GEOS-1')

In [ ]:
geos1_data.drop(geos1_data.index[:90], inplace=True)
geos2_data.drop(geos2_data.index[:90], inplace=True)
geos3_data.drop(geos3_data.index[:90], inplace=True)

Now that we have analyzed the warm-up period and removed these points, let's go ahead and remove the baseline, which is the meadian of each time series. Let's see how much they differ as well.

In [ ]:
print(geos1_data['Bx'].median(), geos2_data['Bx'].median(), geos3_data['Bx'].median())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

We have removed the warm-up period but we still have some issues. This is often because for very large values, linear plots tend to trick us. How would the data look like if it was in a logarithmic plot?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
ax.plot(geos1_data['Bx'], color='#FF9FE5', marker='o', label='GEOS-1')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
ax.plot(geos1_data['Bx'], color='#FF9FE5', marker='o', label='GEOS-1')
ax2 = ax.twinx()
ax2.plot(geos1_data['ctemp'], color='navy', marker='.', label='GEOS-1')
ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax2.set_ylabel('Temperature [C]', fontsize=14)

Temperature correction is beyond the scope of this laboratory exercise. Nominally, you would want to correct for temperature variations by a polynomial fit. Lucky for us, in Alaska, cold winter temperatures act as a first order function. For the sake of the exercise, we will remove the points where temperature drops sharply. 

In [ ]:
geos1_data.drop(geos1_data.index[:200], inplace=True)
geos2_data.drop(geos2_data.index[:200], inplace=True)
geos3_data.drop(geos3_data.index[:200], inplace=True)

After we removed these data points, let's go ahead and reset the index.

In [ ]:
geos1_data.reset_index(drop=True, inplace=True)
geos2_data.reset_index(drop=True, inplace=True)
geos2_data.reset_index(drop=True, inplace=True)

Now let's add the Poker Flat and College site data to this plot. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', label='GEOS-3')

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', label='CMO')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

Before we start analyzing the comparison in more depth there are a few more considerations we need to take into account. The first one is the overlapping of lines. We can introduce the alpha, transparency parameter, to show overlapping lines  better. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

ax.plot(geos1_data['timeString'], geos1_data['Bx']-geos1_data['Bx'].median(), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['Bx']-geos2_data['Bx'].median(), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['Bx']-geos3_data['Bx'].median(), color='#FF858D', alpha=0.6, label='GEOS-3')


ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-1500,1500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=12))

ax.legend(loc=1, fontsize=12)
ax.grid()

Data seems very odd, almost like we put the sensor in the exact opposite direction. How do we fix that?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,0]-np.median(poker_data['thg_mag_pokr']['y'][:,0]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

ax.plot(geos1_data['timeString'], -1*(geos1_data['Bx']-geos1_data['Bx'].median()), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], -1*(geos2_data['Bx']-geos2_data['Bx'].median()), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], -1*(geos3_data['Bx']-geos3_data['Bx'].median()), color='#FF858D', alpha=0.6, label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - X Component', fontsize=16)

ax.set_ylim([-1500,500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))

ax.legend(loc=3, fontsize=12)
ax.grid()

fig.savefig('./magnetometer_data_comparison_bx_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

Before we move onto other components, let's try to answer some of these questions.
1. How many substorm like signatures can we count?
2. How long is the duration of a typical substorm?
3. Is there a time of day we see these signatures more often?
4. What is the largest magnetic field perturbation we see?
5. What is the most geomagnetically active day?

What are some other things you notice?

Now let's check out how other components look like.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,1]-np.median(poker_data['thg_mag_pokr']['y'][:,1]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,1]-np.median(college_data['thg_mag_cmo']['y'][:,1]), color='#77C77C', alpha=0.6, label='CMO')

ax.plot(geos1_data['timeString'], geos1_data['By']-geos1_data['By'].median(), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], geos2_data['By']-geos2_data['By'].median(), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], geos3_data['By']-geos3_data['By'].median(), color='#FF858D', alpha=0.6, label='GEOS-3')

ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - Y Component', fontsize=16)

ax.set_ylim([-1000,500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))

ax.legend(loc=3, fontsize=12)
ax.grid()

fig.savefig('./magnetometer_data_comparison_by_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,5))
ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,2]-np.median(poker_data['thg_mag_pokr']['y'][:,2]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,2]-np.median(college_data['thg_mag_cmo']['y'][:,2]), color='#77C77C', alpha=0.6, label='CMO')

ax.plot(geos1_data['timeString'], (geos1_data['Bz']-geos1_data['Bz'].median()), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], (geos2_data['Bz']-geos2_data['Bz'].median()), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], (geos3_data['Bz']-geos3_data['Bz'].median()), color='#FF858D', alpha=0.6, label='GEOS-3')


ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - Z Component', fontsize=16)

ax.set_ylim([-1000,500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))

ax.legend(loc=3, fontsize=12)
ax.grid()

#fig.savefig('./magnetometer_data_comparison_bz_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

The z-component also seems to be upside-down. This is not a simple orientation error. Checking with the JHU/APL, it is actually an erroneous sensor. 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12,5))
ax.plot(poker_time_array, poker_data['thg_mag_pokr']['y'][:,2]-np.median(poker_data['thg_mag_pokr']['y'][:,2]), color='#2B50AA', alpha=0.6, label='POKR')
ax.plot(college_time_array, college_data['thg_mag_cmo']['y'][:,2]-np.median(college_data['thg_mag_cmo']['y'][:,2]), color='#77C77C', alpha=0.6, label='CMO')

ax.plot(geos1_data['timeString'], -1*(geos1_data['Bz']-geos1_data['Bz'].median()), color='#FF9FE5', alpha=0.6, label='GEOS-1')
ax.plot(geos2_data['timeString'], -1*(geos2_data['Bz']-geos2_data['Bz'].median()), color='#6E0B34', alpha=0.6, label='GEOS-2')
ax.plot(geos3_data['timeString'], -1*(geos3_data['Bz']-geos3_data['Bz'].median()), color='#FF858D', alpha=0.6, label='GEOS-3')


ax.set_ylabel('Magnetic Field [nT]', fontsize=14)
ax.set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
ax.set_title('Comparison of the Magnetic Field Measurements - Z Component', fontsize=16)

ax.set_ylim([-1000,500])
# Set the date format on the x-axis
date_format = mdates.DateFormatter('%d/%m %H:%M')
ax.xaxis.set_major_formatter(date_format)

# Optionally, you can set the locator for the ticks
ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))

ax.legend(loc=3, fontsize=12)
ax.grid()

fig.savefig('./magnetometer_data_comparison_bz_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

### 4.Investigating the spatial extent of the geomagnetic perturbations

####    Comparison along a similar longitude
Let's start with determining the spatial extent by comparing our magnetometer results with USGS magnetometers in Alaska. This will help us determine the latitudinal extent of the geomagnetic disturbances. Remember this is about how far the center of the ionospheric overhead current is. We will use the 'BRW' and 'SHU' sites.

In [ ]:
barrow_data = pyspedas.themis.gmag(trange=trange, sites=['brw'], 
                                level='l2', time_clip=True,notplot=True)


barrow_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in barrow_data['thg_mag_brw']['x']]

In [ ]:
shumagin_data = pyspedas.themis.gmag(trange=trange, sites=['shu'], 
                                level='l2', time_clip=True,notplot=True)

shumagin_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in shumagin_data['thg_mag_shu']['x']]

Now let's plot the data.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10,8))

axes[0].plot(barrow_time_array, barrow_data['thg_mag_brw']['y'][:,0]-np.median(barrow_data['thg_mag_brw']['y'][:,0]), color='navy', alpha=0.6, label='BRW')


axes[1].plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

axes[1].plot(geos1_data['timeString'], -1*(geos1_data['Bx']-geos1_data['Bx'].median()), color='#FF9FE5', alpha=0.6, label='GEOS-1')
axes[1].plot(geos2_data['timeString'], -1*(geos2_data['Bx']-geos2_data['Bx'].median()), color='#6E0B34', alpha=0.6, label='GEOS-2')
axes[1].plot(geos3_data['timeString'], -1*(geos3_data['Bx']-geos3_data['Bx'].median()), color='#FF858D', alpha=0.6, label='GEOS-3')

axes[2].plot(shumagin_time_array, shumagin_data['thg_mag_shu']['y'][:,0]-np.median(shumagin_data['thg_mag_shu']['y'][:,0]), color='indigo', alpha=0.6, label='SHU')

for ax in axes:
    ax.set_ylabel('$B_{X}$ [nT]', fontsize=14)
    
    ax.set_ylim([-1500,500])
    ax.xaxis.set_major_formatter(date_format)
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))
    ax.legend(loc=3, fontsize=12)
    ax.grid()

axes[2].set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
axes[0].set_title('Latitudinal Comparison of the Magnetic Field Measurements - X Component', fontsize=16)


fig.savefig('./latitudinal_magnetometer_data_comparison_bx_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

What can we tell about the ionospheric current system? Can we tell where is the center of the overhead current?

#### Comparison along similar latitudes

Now let's look at the longitudinal extent of these perturbations by comparing the measurements with other magnetometer across the globe which are deployed in similar latitudes to Fairbanks, AK. We are going to use Leirvogur ('LRV'; Iceland), Roervik ('RVK'; Norway) and Baker Lake ('BLC'; Canada).

In [ ]:
leirvogur_data = pyspedas.themis.gmag(trange=trange, sites=['lrv'], 
                                level='l2', time_clip=True,notplot=True)

leirvogur_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in leirvogur_data['thg_mag_lrv']['x']]

In [ ]:
roervik_data = pyspedas.themis.gmag(trange=trange, sites=['rvk'], 
                                level='l2', time_clip=True,notplot=True)

roervik_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in roervik_data['thg_mag_rvk']['x']]

In [ ]:
baker_data = pyspedas.themis.gmag(trange=trange, sites=['blc'], 
                                level='l2', time_clip=True,notplot=True)

baker_time_array = [dt.datetime(1970, 1, 1) + dt.timedelta(seconds=seconds) for seconds in baker_data['thg_mag_blc']['x']]

Now let's plot the data.

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(10,10))



axes[0].plot(college_time_array, college_data['thg_mag_cmo']['y'][:,0]-np.median(college_data['thg_mag_cmo']['y'][:,0]), color='#77C77C', alpha=0.6, label='CMO')

axes[0].plot(geos1_data['timeString'], -1*(geos1_data['Bx']-geos1_data['Bx'].median()), color='#FF9FE5', alpha=0.6, label='GEOS-1')
axes[0].plot(geos2_data['timeString'], -1*(geos2_data['Bx']-geos2_data['Bx'].median()), color='#6E0B34', alpha=0.6, label='GEOS-2')
axes[0].plot(geos3_data['timeString'], -1*(geos3_data['Bx']-geos3_data['Bx'].median()), color='#FF858D', alpha=0.6, label='GEOS-3')

axes[1].plot(baker_time_array, baker_data['thg_mag_blc']['y'][:,0]-np.median(baker_data['thg_mag_blc']['y'][:,0]), color='mediumvioletred', label='BLC')
axes[2].plot(leirvogur_time_array, leirvogur_data['thg_mag_lrv']['y'][:,0]-np.median(leirvogur_data['thg_mag_lrv']['y'][:,0]), color='indigo', label='LRV')
axes[3].plot(roervik_time_array, roervik_data['thg_mag_rvk']['y'][:,0]-np.median(roervik_data['thg_mag_rvk']['y'][:,0]), color='royalblue', label='RVK')

for ax in axes:
    ax.set_ylabel('$B_{X}$ [nT]', fontsize=14)
    
    ax.set_ylim([-1500,500])
    ax.xaxis.set_major_formatter(date_format)
    ax.xaxis.set_major_locator(mdates.HourLocator(interval=36))
    ax.legend(loc=3, fontsize=12)
    ax.grid()

axes[3].set_xlabel('Universal Time [D/M HH:MM]', fontsize=14)
axes[0].set_title('Longitudinal Comparison of the Magnetic Field Measurements - X Component', fontsize=16)


fig.savefig('./longitudinal_magnetometer_data_comparison_bx_{0:%Y_%m_%d_%H}.png'.format(geos1_data['timeString'].iloc[0]), bbox_inches='tight', dpi=200)

Let's try to answer the following questions.
1. Which intervals are recorded as geomagnetically disturbed in all four magnetometer stations?
2. Which intervals are localized?
3. Do stations close to each other record the same magnetic signatures?